In [1]:
!pip install asyncpraw 

In [497]:
import pandas as pd

In [599]:
review_keywords = (
    r"(good service|bad service|friendly staff|rude crew|helpful|unhelpful|"
    r"flew|fly|flown|took a flight|onboard|aircraft|seating|legroom|"
    r"on time|delayed|canceled|missed flight|"
    r"comfortable|cramped|smooth flight|bumpy|turbulence|"
    r"cheap ticket|expensive|affordable|overpriced|"
    r"baggage|lost luggage|extra charge|carry-on|"
    r"good experience|bad experience|would recommend|never again|worst airline)"
)
unrelated_keywords = (
    r"(Thai Airways|other airline|different airline|another airline|better airline|\?$"
    r"|but people here aren't really shy about checking stuff|Cathay Pacific|IATA|hand carry rule|"
    r"OP|That’s a great itinerary|scoot|Fernandes|Maayos|Thanks|pyramid scheme)"
)

In [601]:
import asyncpraw
import asyncio
import re
import nest_asyncio

async def getAirAsiaReviews(url):
    reddit = asyncpraw.Reddit(
        client_id="", 
        client_secret="", 
        user_agent="", 
        username="",
        password=""
    )

    try:
        submission = await reddit.submission(url=url)
        await submission.comments.replace_more(limit=0)  # Load all comments

        airasia_reviews = []
        all_comments = submission.comments.list()

        for comment in all_comments:
            if isinstance(comment, asyncpraw.models.Comment):
                if not re.search(unrelated_keywords, comment.body, re.IGNORECASE):
                    airasia_reviews.append(comment.body)

        return airasia_reviews

    except asyncpraw.exceptions.APIException as e:
        print(f"API Error: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")
    return []

In [603]:
async def processUrls(url_list):
    all_reviews = []
    
    tasks = [getAirAsiaReviews(url) for url in url_list]
    results = await asyncio.gather(*tasks)

    for reviews in results:
        all_reviews.extend(reviews)  

    print(f"Total Comments: {len(all_reviews)}")  
    for review in all_reviews:
        print("-" * 20)
        print(review)

    await reddit.close()

In [605]:
urls = [
    "https://www.reddit.com/r/malaysia/comments/yiauxn/whats_your_opinion_on_airasia/",
    "https://www.reddit.com/r/travel/comments/1fr6bdo/experience_with_airasia/",
]

In [607]:
await processUrls(urls)

Total Comments: 43
--------------------
Basically a legal scam company.
--------------------
Never flying them again. Tony still owes me RM500
--------------------
[deleted]
--------------------
Better late than never. 

Good chance of flight being delayed especially during peak season..

However I have so far not.been into an aircraft that felt outdated or needed repairs. So at least safety is still a priority for them.
--------------------
Not sure about you guys. I flew with them every month back and forth since Feb this year. Not even once my flight got delayed. Maybe delay happen on certain flight, but no problem so far for me. It is the same precovid.
--------------------
There's a 50/50 chance that your flight is going to be rescheduled or cancelled. Just need to have backup plans.
--------------------
Having flown with dozens of airlines from around the world be it budget or full service ones, AA is probably the worst in the world in areas of service and back-up support. You ca